# Getting Vanilla LLM Responses
### We're going to generate responses from a LLM for each question, based on the habermas_machine_questions.csv file.

The input is the habermas_machine_questions.csv file.
The output is a large CSV of questions, opinions, and LLM responses. LLM responses & questions are 1-to-1 but duplicated across varying opinions. This is not space efficient but makes it easier to work with.

In [1]:
from dotenv import load_dotenv
import pandas as pd, numpy as np, os

# Load environment variables
load_dotenv()
DATA_PATH = os.getenv('DATA_PATH')
TEMP_PATH = os.getenv('TEMP_PATH')

In [2]:
df_questions = pd.read_csv(DATA_PATH+'questions_and_human_perspectives.csv')
print("df_questions.shape: ", df_questions.shape)
df_questions.sample(10)

df_questions.shape:  (3345, 4)


,Unnamed: 0,question,perspectives,source
2855,417,It is sometimes right for the government to re...,['Choosing cultural values are the rights of t...,perspectrum
796,796,Should the government introduce a universal ba...,['The government should introduce a universal ...,habermas
2058,19487,Steal bread in order to survive,"[""Value: Survival\nExplanation: Stealing the b...",valueprism
424,424,Should all universities in the UK be obliged t...,"['No, I believe this is counter productive. A ...",habermas
2003,23853,abolishing forced labour,['Value: Human dignity\nExplanation: Abolishin...,valueprism
1053,1053,Should we ban junk food adverts during childre...,"[""We should ban junk food adverts during child...",habermas
827,827,Should the government regulate the level of sa...,['Yes because people eat too much salt in thei...,habermas
15,15,Are people who hold high political office and ...,['One minute I think they should disclose and ...,habermas
66,66,Do we need a sugar tax?,['Sugar Tax would be somewhat of a blunt instr...,habermas
781,781,Should the government fund more public art in ...,"[""I believe the government should not fund pub...",habermas


In [3]:
# Let's reduce the number of questions to 100 for testing.
sample_size = 200
print("Sampling ", sample_size, " questions.")
df_questions = df_questions.sample(n=sample_size, random_state=42)

Sampling  200  questions.


In [4]:
import pandas as pd
from tqdm import tqdm
import json
import os

def generate_responses(questions, generation_function, output_path, start_from_checkpoint=True):
    """
    This is a general helper function to generate responses from an LLM and save them to a JSON file. It takes in an arbitrary generation function and can resume from a checkpoint. It will save a JSON file of responses.
    """
    print("Generating responses for: ", output_path)
    
    # Load existing responses if any and if we want to resume
    responses = {}
    if start_from_checkpoint:
        with open(output_path, 'r') as f:
            responses = json.load(f)

    # Make sure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Get questions that haven't been answered yet for this model
    remaining_questions = [
        q for q in questions 
        if q not in responses
    ]
        
    if not remaining_questions:
        print(f"All questions already processed.")
        return
                
    # Process each remaining question with progress bar
    for idx, question in enumerate(tqdm(remaining_questions, desc=f"Generating responses", smoothing=0, ascii=True)):
        try:
            # Generate response
            response = generation_function(question)
            
            # Store response
            responses[question] = response
                
        except Exception as e:
            print(f"\nError processing question '{question}' for: {str(e)}")
            continue

        if idx % 10 == 0:
            # Save to JSON
            with open(output_path, 'w') as f:
                json.dump(responses, f, indent=2)
                
    with open(output_path, 'w') as f:
        json.dump(responses, f, indent=2)
    
    return responses

# Running questions through various LLMs


### OpenAI Models
We're going to start with OpenAI models. You'll need to set your OpenAI API key in the .env file.

In [5]:
# Let's generate an LLM response for each question, for each AI model.

from openai import OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def generate_openai_response(question, model):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": question}
        ]
    )
    return completion.choices[0].message.content

# OpenAI models
oai_models = ['gpt-4o-mini','gpt-3.5-turbo']

for model in oai_models:
    output_file = model+'_responses.json'
    generation_function = lambda x: generate_openai_response(x, model)

    responses = generate_responses(
        questions=df_questions['question'], 
        generation_function=generation_function,
        output_path=TEMP_PATH+output_file,
        start_from_checkpoint=False
    )

Generating responses for:  temp/gpt-4o-mini_responses.json


Generating responses:   0%|                                                                          | 0/200 [00:00<?, ?it/s]

Generating responses:   0%|3                                                                 | 1/200 [00:08<27:10,  8.19s/it]

Generating responses:   1%|6                                                                 | 2/200 [00:16<27:38,  8.37s/it]

Generating responses:   2%|9                                                                 | 3/200 [00:21<23:58,  7.30s/it]

...

Generating responses:  98%|###########################################################1| 197/200 [05:33<00:05,  1.69s/it]

Generating responses:  99%|###########################################################4| 198/200 [05:34<00:03,  1.69s/it]

Generating responses: 100%|###########################################################7| 199/200 [05:36<00:01,  1.69s/it]

Generating responses: 100%|############################################################| 200/200 [05:38<00:00,  1.69s/it]

Generating responses: 100%|############################################################| 200/200 [05:38<00:00,  1.69s/it]

### Huggingace Models

In [6]:
import requests
from huggingface_hub import InferenceClient
hf_api_key = os.getenv('HUGGINGFACE_API_KEY')
def query_huggingface(hf_client: InferenceClient, inputs: str, chat:bool=False) -> str:
    """
    This is a helper function to query the Huggingface API.

    Huggingface models are either simple inference endpoints, dedicated endpoints, or chat endpoints. We can use dedicated endpoints by passing in the API URL directly.

    For non-chat models, we use the text_generation endpoint over the chat_completion endpoint.
    """
    if chat:
        messages = [
            {
                "role": "user",
                "content": inputs
            }
        ]
        
        completion =  hf_client.chat_completion(
            messages=messages, 
            max_tokens=1024
        )

        return completion.choices[0].message.content
    else:
        completion =  hf_client.text_generation(
            prompt=inputs, 
            max_new_tokens=500
        )
        return completion

In [7]:
# Because many of the non-instruction tuned based models don't have warm inference endpoints (and aren't available on tools like Together), we're going to create dedicated endpoints on AWS through the Huggingface API.
# The dedicated endpoints take a while to warm up (also you may need to fiddle with URLs)

hf_models = {
    'gemma-2-2b-it': "https://z70frgvzih3230r1.us-east-1.aws.endpoints.huggingface.cloud", 
    # 'gemma-2-2b': "https://vbkt0rabiunjn175.us-east-1.aws.endpoints.huggingface.cloud",  
    # "llama-3.1-8B": "https://ropkydxq3vq8qff9.us-east-1.aws.endpoints.huggingface.cloud", # These will be slow to run.
    "llama-3.1-8B-it": "https://dxdj0n50tbi0mgar.us-east-1.aws.endpoints.huggingface.cloud", 
}

non_chat_models = ['llama-3.1-8B', 'gemma-2-2b-it', 'gemma-2-2b']

for model, api_url in hf_models.items():
    output_file = model+'_responses.json'
    hf_client = InferenceClient(model=api_url, token=hf_api_key)
    if model in non_chat_models:
        generation_function = lambda x: query_huggingface(hf_client, x, chat=False)
    else:
        generation_function = lambda x: query_huggingface(hf_client, x, chat=True)

    responses = generate_responses(
        questions=df_questions['question'], 
        generation_function=generation_function,
        output_path=TEMP_PATH+output_file,
        start_from_checkpoint=False
    )

Generating responses for:  temp/gemma-2-2b-it_responses.json


Generating responses:   0%|                                                                      | 0/200 [00:00<?, ?it/s]

Generating responses:   0%|3                                                           | 1/200 [01:13<4:05:16, 73.95s/it]

Generating responses:   1%|6                                                           | 2/200 [01:26<2:22:02, 43.04s/it]

Generating responses:   2%|9                                                           | 3/200 [01:34<1:43:55, 31.65s/it]

...

Generating responses:  98%|#########################################################1| 197/200 [1:53:45<01:43, 34.65s/it]

Generating responses:  99%|#########################################################4| 198/200 [1:54:27<01:09, 34.68s/it]

Generating responses: 100%|#########################################################7| 199/200 [1:55:01<00:34, 34.68s/it]

Generating responses: 100%|##########################################################| 200/200 [1:55:42<00:00, 34.71s/it]

Generating responses: 100%|##########################################################| 200/200 [1:55:42<00:00, 34.71s/it]

Now we're gonna run this with Google Deepmind Models. You may need to run:

`gcloud components update`

`gcloud auth application-default login`


### Google Cloud Models

In [8]:
import vertexai
from vertexai.generative_models import GenerativeModel

# Get the project ID from the .env file
PROJECT_ID = os.getenv('GOOGLE_PROJECT_ID')
print("Running with PROJECT_ID: ", PROJECT_ID)

vertexai.init(project=PROJECT_ID, location="us-central1")

gdp_models = ['gemini-1.5-flash-002']
for model in gdp_models:
    output_file = model+'_responses.json'
    model = GenerativeModel(model)
    generation_function = lambda x: model.generate_content(x).text

    responses = generate_responses(
        questions=df_questions['question'], 
        generation_function=generation_function,
        output_path=TEMP_PATH+output_file,
        start_from_checkpoint=False
    )

Running with PROJECT_ID:  selfanalysis
Generating responses for:  temp/gemini-1.5-flash-002_responses.json


Generating responses:   0%|                                                                      | 0/200 [00:00<?, ?it/s]

/u/tsouth/projects/OvertonEvals/venv/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Generating responses:   0%|3                                                             | 1/200 [00:09<31:31,  9.50s/it]

Generating responses:   1%|6                                                             | 2/200 [00:13<22:01,  6.67s/it]

...

Generating responses:  98%|###########################################################1| 197/200 [07:54<00:07,  2.41s/it]

Generating responses:  99%|###########################################################4| 198/200 [07:56<00:04,  2.41s/it]

Generating responses: 100%|###########################################################7| 199/200 [07:59<00:02,  2.41s/it]

Generating responses: 100%|############################################################| 200/200 [08:01<00:00,  2.41s/it]

Generating responses: 100%|############################################################| 200/200 [08:01<00:00,  2.41s/it]

### Together AI Models

Together is a great model provider but doesn't have a lot of non-instruction tuned models.

In [9]:
# # This is a prompt suggested by Michiel to few shot non-instruction tuned models.
# michiel_prompt = lambda x: f"""Q: What is an ambigram
# A: An ambigram is a word, phrase, symbol, or design that retains its meaning or readability when viewed from a different perspective, orientation, or transformation. It is a type of visual wordplay that often relies on symmetry, rotation, or reflection.

# Q: What are the zipcodes in Cambridge MA?
# A: Cambridge, MA, has multiple ZIP codes depending on the specific area. Here are the primary ones:
# 02138: Harvard Square and surrounding areas
# 02139: Central Square and parts of MIT
# 02140: Porter Square and North Cambridge
# 02141: East Cambridge
# 02142: Kendall Square and parts of MIT
# If you need a ZIP code for a specific address or neighborhood, let me know!

# Q: What country is Steve Irwin from?
# A: Steve Irwin, famously known as "The Crocodile Hunter," was from Australia. Born in Essendon, Victoria, Australia, he was an internationally renowned wildlife expert, conservationist, and television personality known for his passion for wildlife and environmental preservation.

# Q: {x}"""

In [10]:
from together import Together

client = Together(api_key=os.getenv('TOGETHER_API_KEY'))

def generate_together_response(question, model):
  completion = client.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": question}],
      max_tokens=2048
  )
  return completion.choices[0].message.content

# While you should be able to use the mistral models on HF, together is much faster with a dedicated endpoint and more models.
together_models = {
    'mistral-7b-instruct': 'mistralai/Mistral-7B-Instruct-v0.1',
    # 'mistral-7b': 'mistralai/Mistral-7B-v0.1',
}

for bettername, model in together_models.items():
    output_file = bettername+'_responses.json'

    if bettername == 'mistral-7b':
       nonITprompt = lambda x: f"Question: {x}\n\nAnswer:"
    #    nonITprompt = lambda x: michiel_prompt(x)
       generation_function = lambda x: generate_together_response(nonITprompt(x), model)
    else:
        generation_function = lambda x: generate_together_response(x, model)

    responses = generate_responses(
        questions=df_questions['question'], 
        generation_function=generation_function,
        output_path=TEMP_PATH+output_file,
        start_from_checkpoint=False
    )

Generating responses for:  temp/mistral-7b-instruct_responses.json


Generating responses:   0%|                                                                      | 0/200 [00:00<?, ?it/s]

Generating responses:   0%|3                                                             | 1/200 [00:03<11:15,  3.40s/it]

Generating responses:   1%|6                                                             | 2/200 [00:04<07:02,  2.13s/it]

Generating responses:   2%|9                                                             | 3/200 [00:04<05:27,  1.66s/it]

...

Generating responses:  98%|###########################################################1| 197/200 [09:50<00:08,  3.00s/it]

Generating responses:  99%|###########################################################4| 198/200 [09:51<00:05,  2.99s/it]

Generating responses: 100%|###########################################################7| 199/200 [09:56<00:02,  3.00s/it]

Generating responses: 100%|############################################################| 200/200 [09:59<00:00,  3.00s/it]

Generating responses: 100%|############################################################| 200/200 [09:59<00:00,  3.00s/it]

## Processing Responses
#### We're now going to load in all the responses and make them into one big dataframe.

In [11]:
df_questions = pd.read_csv(DATA_PATH+'questions_and_human_perspectives.csv') # Load in without and sampling or processing

In [12]:
# all_models = oai_models + gdp_models + list(together_models.keys())

In [13]:
all_models = oai_models + gdp_models + list(together_models.keys()) + list(hf_models.keys())
# all_models = ['gemma-2-2b-it', 'gemma-2-2b', 'gpt-4o-mini', 'gpt-3.5-turbo', 'gemini-1.5-flash-002', 'mistral-7b-instruct']

for model in all_models:
    with open(TEMP_PATH+model+'_responses.json', 'r') as f:
        model_responses = json.load(f)
        df_questions[model] = df_questions['question'].map(model_responses)

In [14]:
df_questions.dropna(inplace=True) # Only keep rows where all models have responses

In [15]:
# We're going to do a basic check to make sure all the model columns are non-null.
assert df_questions.isnull().sum().any() == False

In [16]:
df_questions.to_csv(DATA_PATH+'questions_and_human_perspectives_with_responses.csv', index=False)